In [2]:
from nltk.corpus import stopwords
import warnings
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer #for creating count vectors
from sklearn.metrics.pairwise import cosine_similarity #cosine similarity calculator
import nltk
from nltk.tokenize import word_tokenize
import gensim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

C:\Users\elandman\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import feedparser as fp
import newspaper
from newspaper import Article
import time
from time import mktime
from datetime import datetime
from datetime import date
import pandas as pd
import json
import pprint
import dateutil

#### 1 Website data ####

## 1A ##  From JSON file - for final version

with open('NewsPapers.json') as data_file: #Loads the JSON files with news URLs
    companies = json.load(data_file)

## 1B ## From variable - this is for testing, makes it way faster
#website = {"cnn": {"link": "http://edition.cnn.com/", "rss": "http://rss.cnn.com/rss/cnn_topstories.rss"},
#          "cnbc":{"link": "https://www.cnbc.com/", "rss": "https://www.cnbc.com/id/10000664/device/rss/rss.html"}}
#          "bbc": {"rss": "http://feeds.bbci.co.uk/news/rss.xml","link": "http://www.bbc.com/"},
#          "theguardian": {"rss": "https://www.theguardian.com/uk/rss", "link": "https://www.theguardian.com/international"},
#          "washingtonpost": {"rss": "http://feeds.washingtonpost.com/rss/world", "link": "https://www.washingtonpost.com/"},
#          "reuters": {"rss": "http://feeds.reuters.com/reuters/businessNews"},
#          "fortune": {"rss": "https://fortune.com/feed"}}


#### 2 Todays date - for filtering the articles ####
today = str(date.today()) 
print("Today's date:", today)


#### 3 Scraping the news articles ####

text_list = []
source_list = []
article_list = []
date_list = []
time_list = []
title_list = []

for source, value in companies.items():
        d = fp.parse(value['rss']) #if there is an RSS value for a company in the website data, it will be extracted into d
        article={}
        
        for entry in d.entries:
            if hasattr(entry, 'published'):
                article['source'] = source
                source_list.append(article['source'])
                
                #getting the article URLs
                article['link'] = entry.link
                article_list.append(article['link'])
                
                #getting the article published dates
                date = (getattr(entry, 'published'))
                date = dateutil.parser.parse(date)
                date_formated = date.strftime("%Y-%m-%d")
                time_formated = date.strftime("%H:%M:%S %Z") #hour, minute, timezone (converted)
                date_list.append(date_formated)
                time_list.append(time_formated)
                
                #getting the titles
                content = Article(entry.link)
                try:
                    content.download() #downloading article content
                    #downloading takes approx. 3min to load
                    content.parse()                    
                except Exception as e: 
                    #in case the download fails, it prints the error and immediatly continues with downloading the next article
                    print(e)
                    print("continuing...")
                title = content.title #extract article titles
                title_list.append(title)
                text = content.text
                text_list.append(text)
                
#creating dicts for formatting and inserting to pandas df
source_dict = {'source':source_list}
link_dict = {'link':article_list}
date_dict = {'published_date':date_list}
time_dict = {'published_time':time_list}
title_dict = {'title':title_list}

#creating separate pandas dfs for each feature
source_df = pd.DataFrame(source_dict, index=None)
link_df = pd.DataFrame(link_dict, index=None)
date_df = pd.DataFrame(date_dict, index=None)
time_df = pd.DataFrame(time_dict, index=None)
title_df = pd.DataFrame(title_dict, index=None)

#join all pandas dfs together
news_df = source_df.join(link_df)
news_df = news_df.join(date_df)
news_df = news_df.join(time_df)
news_df = news_df.join(title_df)

#after 4,5 min, pandas DF sould be created with link, published_date, published_time, title and text
#4.5min
#with text 4min


Today's date: 2019-11-11


KeyboardInterrupt: 

In [254]:
text_dict = {'text':text_list}

#creating separate pandas dfs for each feature
text_df = pd.DataFrame(text_dict, index=None)

#join all pandas dfs together
news_df = news_df.join(text_df)

In [2]:
stop = stopwords.words('english')

In [260]:
warnings.filterwarnings("ignore")

In [296]:
values = (list(source_dict.values()))
for i in values:
    x = set(i)
c = dict.fromkeys(x, "")
print(c)

news_df_daily = news_df[news_df.published_date == today] #dataframe filtered by todays date
news_df_daily = news_df_daily.reset_index(drop=True)

news_df_daily["clean_title"] = news_df_daily["title"].str.replace('[^\w\s]','') #remove punctuation from titles
news_df_daily["clean_title"] = news_df_daily["clean_title"].str.replace('\d+', '') #remove numbers
news_df_daily["clean_title"] = news_df_daily["clean_title"].str.lower() #make titles lower case

news_df_daily["clean_text"] = news_df_daily["text"].str.lower() #make titles lower case
news_df_daily["clean_text"] = news_df_daily["clean_text"].str.replace('[^\w\s]','')
news_df_daily["clean_text"] = news_df_daily["clean_text"].replace(c, regex=True)



#news_df_daily["clean_text"] = news_df_daily["clean_text"].str.replace('[^\w\s]','')

df_updated = news_df_daily #create df only with cleaned titles
df_updated = pd.DataFrame(df_updated) #defining as df

#### this is for splitting the titles in separate columns, in case needed some day ####

#split = df_updated.str.split(expand=True)
#title_splitted = pd.DataFrame(split)
#title_splitted

#### end ####

#filter out the stopwords
df_updated['clean_title'] = df_updated['clean_title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_updated

{'google business analytics': '', 'bbc': '', 'buzzfeed': '', 'google news': '', 'fortune': '', 'theguardian': '', 'yahoonews': '', 'aljazeera': '', 'washingtonpost': '', 'seekingalpha': '', 'hubspot': '', 'reuters': '', 'investing': '', 'economictimes': '', 'timesofindia': '', 'cnbc': '', 'nytimes': '', 'cnn': ''}


,source,link,published_date,published_time,title,text,clean_title,clean_text
0,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/_L...,2019-11-10,19:25:27 UTC,'You have to earn votes and not buy them': Som...,Washington (CNN) Democratic presidential candi...,earn votes buy dems wary michael bloombergs po...,washington democratic presidential candidate ...
1,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/k1...,2019-11-10,17:54:20 UTC,Amy Klobuchar: I certainly welcome Michael Blo...,"During an interview with CNN's Jake Tapper , D...",amy klobuchar certainly welcome michael bloomb...,during an interview with s jake tapper democr...
2,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/3h...,2019-11-10,16:00:40 UTC,A fractured Senate looks to the past for impea...,(CNN) As the impeachment inquiry of President ...,fractured senate looks past impeachment trial ...,as the impeachment inquiry of president donal...
3,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/og...,2019-11-10,14:25:24 UTC,Tom Steyer to appear at CNN town hall,Washington (CNN) Democratic presidential candi...,tom steyer appear cnn town hall,washington democratic presidential candidate ...
4,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/Qe...,2019-11-10,15:38:27 UTC,The Joker's warning for Trump and Democrats,Sign up to get our new weekly column as a news...,jokers warning trump democrats,sign up to get our new weekly column as a news...
5,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/0j...,2019-11-10,19:33:33 UTC,Washington Post: Nikki Haley says top Trump ai...,Washington (CNN) Former United Nations ambassa...,washington post nikki haley says top trump aid...,washington former united nations ambassador n...
6,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/ie...,2019-11-10,13:59:37 UTC,Why these people are walking 230 miles to the ...,(CNN) Eliana Fernández has blisters on her fee...,people walking miles supreme court,eliana fernández has blisters on her feet and...
7,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/5r...,2019-11-10,16:14:16 UTC,DACA heads to the Supreme Court,"A looming government shutdown on the horizon, ...",daca heads supreme court,a looming government shutdown on the horizon t...
8,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/N3...,2019-11-10,14:19:38 UTC,A Russian professor was pulled from a river. P...,Moscow (CNN) Russian law enforcement agencies ...,russian professor pulled river police said wom...,moscow russian law enforcement agencies are q...
9,cnn,http://rss.cnn.com/~r/rss/cnn_topstories/~3/94...,2019-11-10,02:14:40 UTC,'Baby Trump' balloon at Alabama-LSU football g...,"(CNN) The ""Baby Trump"" balloon has been slashe...",baby trump balloon alabamalsu football game sl...,the baby trump balloon has been slashed and ...
